Scenario 1: Query to get who are getting equal salary

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.appName("Scenario-app").getOrCreate()

In [ ]:
data = [("001", "Monika", "Arora", 100000, "2014-02-20 09:00:00", "HR"),("002", "Niharika", "Verma", 300000, "2014-06-11 09:00:00", "Admin"),("003", "Vishal", "Singhal", 300000, "2014-02-20 09:00:00", "HR"),("004", "Amitabh", "Singh", 500000, "2014-02-20 09:00:00", "Admin"),("005", "Vivek", "Bhati", 500000, "2014-06-11 09:00:00", "Admin")]

myschema = ["workerid","firstname","lastname","salary","joiningdate","depart"]

In [ ]:
df1 = spark.createDataFrame(data, myschema)
df1.show()
df1.groupBy("depart").count().show()
df1.groupBy("depart").agg(count('*')).show()

In [ ]:
t1 = df1.alias("t1")
t2 = df1.alias("t2")
df_joined = t1.join(
    t2,
    (t1["workerid"] != t2["workerid"]) & (t1["salary"] == t2["salary"]),
    "inner"
)

df_joined.select("t1.firstname").show()



Scenario 2: (Need the dates when the status gets changed like ordered to dispatched)

In [ ]:
data = [
      (1, "1-Jan", "Ordered"),
      (1, "2-Jan", "dispatched"),
      (1, "3-Jan", "dispatched"),
      (1, "4-Jan", "Shipped"),
      (1, "5-Jan", "Shipped"),
      (1, "6-Jan", "Delivered"),
      (2, "1-Jan", "Ordered"),
      (2, "2-Jan", "dispatched"),
      (2, "3-Jan", "shipped")]
myschema = ["orderid","statusdate","status"]

In [ ]:
df2 = spark.createDataFrame(data,myschema)
df2.show()

In [ ]:
df2.createOrReplaceTempView("orders")

In [ ]:
df2_result = spark.sql("SELECT * from orders where status='dispatched' and orderid in (select orderid from orders where status='Ordered')")
df2_result.show()

Scenario 3: difference from next order

In [ ]:
data = [(1111, "2021-01-15", 10),
        (1111, "2021-01-16", 15),
        (1111, "2021-01-17", 30),
        (1112, "2021-01-15", 10),
        (1112, "2021-01-15", 20),
        (1112, "2021-01-15", 30)]

myschema = ["sensorid", "timestamp", "values"]

In [ ]:
df3 = spark.createDataFrame(data, myschema)
df3.show()

In [ ]:
from pyspark.sql.window import Window
from pyspark.sql.functions import *

wspec = Window.partitionBy("sensorid").orderBy("values")

final_df = df3.withColumn('newValue', lead("values", 1).over(wspec)) \
            .filter(col('newValue').isNotNull()) \
            .withColumn('values', expr("newValue - values")) \
            .drop(col('newValue')) \
            .orderBy('sensorid')

final_df.show()

scenario 4
user third transaction

user_id	    spend	transaction_date
111	        100.50	01/08/2022 12:00:00
111	        55.00	01/10/2022 12:00:00
121	        36.00	01/18/2022 12:00:00
145	        24.99	01/26/2022 12:00:00
111	        89.60	02/05/2022 12:00:00

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.window import Window

data = [('111','100.50','01/08/2022 12:00:00'),('111','55.00','01/10/2022 12:00:00'),('121','36.00','01/18/2022 12:00:00'),('145','24.99','01/26/2022 12:00:00'),('111','89.60','02/05/2022 12:00:00')]
columns = ['user_id','spend','transaction_date']

df4 = spark.createDataFrame(data, columns)

df4 = df4.withColumn("spend", col("spend").cast("float")) \
        .withColumn("transaction_date", to_date(col("transaction_date"), "MM/dd/yyyy HH:mm:ss"))

w_spec = Window.partitionBy(col('user_id')).orderBy(col('transaction_date').desc())

df4 = df4.withColumn("transaction_no", row_number().over(w_spec)).filter(col('transaction_no')==3)

df4.printSchema()
df4.show()


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("fillna_example").getOrCreate()

data = [
    ("Alice", None, 30),
    ("Bob", 25, None),
    (None, 35, 40),
    ("Charlie", 40, 50)
]
columns = ["Name", "Age", "Score"]
df5 = spark.createDataFrame(data, columns)

# Fill 'Name' with 'Unknown', 'Age' with 0, and 'Score' with the mean score
mean_score = df5.select("Score").agg({"Score": "avg"}).collect()[0][0]
print(mean_score)
df_filled_different = df5.fillna({"Name": "Unknown", "Age": 0, "Score": mean_score})
df_filled_different.show()

In [ ]:

data = [("John,Doe",), ("Jane,Smith",), ("Peter,Jones",), ("Alice,Schultz",)]
df6 = spark.createDataFrame(data, ["FullName"])
df6.show()

In [ ]:

df6 = df6.withColumn('firstName', split(col('FullName'), ",")[0]) \
       .withColumn('lastName', split(col('FullName'), ",")[1])

df6.show()

In [ ]:
data = [(1,"John,Doe",), (1,"Jane,Smith",), (2,"Peter,Jones",), (2,"Alice,Schultz",)]
df = spark.createDataFrame(data, ["gid", "FullName"])
df.show()

In [ ]:
result_df = df.select("gid", explode(split(col("FullName"),",")).alias("splt"))
result_df.show()

In [ ]:
new_df = result_df.groupBy("gid").agg(collect_list(col("splt")).alias("cl"))
result_new_df = new_df.select("gid", concat_ws(",", col("cl")))
result_new_df.show()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, lead, col, when, count, lit, max as max_

spark = SparkSession.builder.getOrCreate()

# Example data
data = [
    (1, "apple"),
    (2, "banana"),
    (3, "carrot"),
    (4, "dates"),
    (5, "eggfruit")
]
df = spark.createDataFrame(data, ["order_id", "item"])

# Step 1️⃣ Create window spec
wspec = Window.orderBy("order_id")

# Step 2️⃣ Add lag and lead columns (equivalent to CTE_1)
cte_1 = df.withColumn("prev_item", lag("item", 1).over(wspec)) \
          .withColumn("next_item", lead("item", 1).over(wspec))

# Step 3️⃣ Find max order_id (equivalent to SELECT COUNT(*) in SQL)
max_order = cte_1.agg(max_("order_id").alias("max_order_id")).collect()[0]["max_order_id"]

# Step 4️⃣ Apply CASE WHEN logic
final_df = cte_1.withColumn(
    "final_item",
    when(col("order_id") % 2 == 0, col("next_item"))
    .when((col("order_id") % 2 != 0) & (col("order_id") == lit(max_order)), col("item"))
    .otherwise(col("prev_item"))
)

final_df.select("order_id", "final_item").show()